# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Google developer API key
from api_keys import g_key
from pprint import pprint
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

cities_df = pd.read_csv("../output_data/cities.csv")
cities_df=cities_df.drop_duplicates(subset=['City_Name','Country_Code'], keep='last')

cities_df

,Unnamed: 0,City_Name,Country_Code,Date,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
1,2,victoria,sc,1619894328,-4.6167,55.4500,82.40,78,40,2.30
3,4,presidencia roque saenz pena,ar,1619894328,-26.7852,-60.4388,84.99,50,0,3.00
4,5,la asuncion,ve,1619894329,11.0333,-63.8628,95.00,49,99,18.41
7,8,stokmarknes,no,1619894330,68.5646,14.9108,33.30,78,100,8.01
8,9,rumoi,jp,1619894330,43.9344,141.6428,38.46,88,100,7.49
...,...,...,...,...,...,...,...,...,...,...
595,596,batagay,ru,1619894492,67.6333,134.6333,28.29,86,100,3.62
596,597,kodiak,us,1619894493,57.7900,-152.4072,39.20,81,1,4.61
597,598,hobbs,us,1619894493,32.7026,-103.1360,70.63,42,20,5.75
598,599,tadpatri,in,1619894494,14.9167,78.0167,85.14,52,4,10.42


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# heatmap 

# Store latitude and longitude in locations
cities_locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"].astype(str).astype(float)

figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
} 

# create figure
fig1 = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(cities_locations, weights=humidity)

#Add layer
fig1.add_layer(heat_layer)

#Display figure
fig1

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# * Narrow down the DataFrame to find your ideal weather condition. For example:
#   * A max temperature lower than 80 degrees but higher than 70.
#   * Wind speed less than 10 mph.
#   * Zero cloudiness.

# Filter the cities with specific conditions and store it in hotel data frame.
hotel_df = cities_df[(cities_df['Temperature'] <= 80 ) & (cities_df['Temperature'] >= 70) & (cities_df['Wind_Speed'] <= 10 ) & (cities_df['Cloudiness'] == 0 )]

# Drop any rows with NULL values
hotel_df.dropna(subset = ['Country_Code'],inplace = True)
hotel_df.head()


,Unnamed: 0,City_Name,Country_Code,Date,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
115,116,nayagarh,in,1619894364,20.1333,85.1000,78.60,69,0,4.27
208,209,pisco,pe,1619894363,-13.7000,-76.2167,70.48,73,0,9.22
220,221,guaruja,br,1619894390,-23.9931,-46.2564,78.06,65,0,8.05
402,403,saint-philippe,re,1619894380,-21.3585,55.7679,75.13,78,0,6.91
474,475,saint-pierre,re,1619894459,-21.3393,55.4781,75.11,78,0,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City_Name,Country_Code,Date,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel Name
115,116,nayagarh,in,1619894364,20.1333,85.1000,78.60,69,0,4.27,
208,209,pisco,pe,1619894363,-13.7000,-76.2167,70.48,73,0,9.22,
220,221,guaruja,br,1619894390,-23.9931,-46.2564,78.06,65,0,8.05,
402,403,saint-philippe,re,1619894380,-21.3585,55.7679,75.13,78,0,6.91,
474,475,saint-pierre,re,1619894459,-21.3393,55.4781,75.11,78,0,5.75,
505,506,le port,re,1619894468,-20.9373,55.2919,75.13,73,0,5.75,
556,557,laguna,br,1619894482,-28.4825,-48.7808,73.00,57,0,1.99,
570,571,atuona,pf,1619894485,-9.8000,-139.0333,78.66,73,0,9.46,


In [6]:
# find the hotel name of each coordinates
# use Google API to get the nearby hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    params = {
    "location": f"{row['Latitude']},{row['Longitude']}",  
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotels",
    "key": g_key
    }

    response = requests.get(base_url, params=params).json()
    try:
        pprint(f"results: {response['results'][0]['name']}")
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        hotel_df.loc[index, 'Lat'] = response['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = response['results'][0]['geometry']['location']['lng']   
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

'results: Trinath Residency'
------------
'results: HOSTAL MIRAMAR'
------------
'results: Hotel Doral Guarujá'
------------
'results: Les Embruns Du Baril'
------------
'results: Le Battant Des Lames'
------------
'results: Lodge Roche Tamarin & Spa Village Nature'
------------
'results: Laguna Tourist Hotel'
------------
'results: Hôtel Hiva Oa Hanakee Lodge'
------------


In [7]:
hotel_df.head()

,Unnamed: 0,City_Name,Country_Code,Date,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel Name,Lat,Lng
115,116,nayagarh,in,1619894364,20.1333,85.1000,78.60,69,0,4.27,Trinath Residency,20.126926,85.105039
208,209,pisco,pe,1619894363,-13.7000,-76.2167,70.48,73,0,9.22,HOSTAL MIRAMAR,-13.727375,-76.222341
220,221,guaruja,br,1619894390,-23.9931,-46.2564,78.06,65,0,8.05,Hotel Doral Guarujá,-23.985241,-46.221868
402,403,saint-philippe,re,1619894380,-21.3585,55.7679,75.13,78,0,6.91,Les Embruns Du Baril,-21.369201,55.727949
474,475,saint-pierre,re,1619894459,-21.3393,55.4781,75.11,78,0,5.75,Le Battant Des Lames,-21.341242,55.461965


In [8]:
# Drop any rows with NULL values
hotel_df.dropna(subset = ['Lat'],inplace = True)
hotel_df.head(9)

,Unnamed: 0,City_Name,Country_Code,Date,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel Name,Lat,Lng
115,116,nayagarh,in,1619894364,20.1333,85.1000,78.60,69,0,4.27,Trinath Residency,20.126926,85.105039
208,209,pisco,pe,1619894363,-13.7000,-76.2167,70.48,73,0,9.22,HOSTAL MIRAMAR,-13.727375,-76.222341
220,221,guaruja,br,1619894390,-23.9931,-46.2564,78.06,65,0,8.05,Hotel Doral Guarujá,-23.985241,-46.221868
402,403,saint-philippe,re,1619894380,-21.3585,55.7679,75.13,78,0,6.91,Les Embruns Du Baril,-21.369201,55.727949
474,475,saint-pierre,re,1619894459,-21.3393,55.4781,75.11,78,0,5.75,Le Battant Des Lames,-21.341242,55.461965
505,506,le port,re,1619894468,-20.9373,55.2919,75.13,73,0,5.75,Lodge Roche Tamarin & Spa Village Nature,-20.949848,55.341256
556,557,laguna,br,1619894482,-28.4825,-48.7808,73.00,57,0,1.99,Laguna Tourist Hotel,-28.465326,-48.769251
570,571,atuona,pf,1619894485,-9.8000,-139.0333,78.66,73,0,9.46,Hôtel Hiva Oa Hanakee Lodge,-9.804012,-139.025696


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City_Name}</dd>
<dt>Country</dt><dd>{Country_Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
# marker_layer 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Create heat layer
heat_layer = gmaps.heatmap_layer(cities_locations, weights=humidity)

figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
} 

fig = gmaps.figure(layout=figure_layout)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='600px'))